In [ ]:
#import library

In [1]:
import sys
import random
import string
import re
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QVBoxLayout, 
                             QComboBox, QLineEdit, QPushButton, QTextEdit, QMessageBox)

In [ ]:
#Password Generation Functionality

In [2]:
# Function to generate password
def generate_password(complexity_level, length):
    if length < 8 or length > 16:
        raise ValueError("Password length should be between 8 and 16 characters.")
    
    lowercase = string.ascii_lowercase
    uppercase = string.ascii_uppercase
    digits = string.digits
    special_characters = string.punctuation

    password = []

    if complexity_level == 'simple':
        characters = lowercase + uppercase
        password.append(random.choice(lowercase))
        password.append(random.choice(uppercase))
    elif complexity_level == 'medium':
        characters = lowercase + uppercase + digits
        password.append(random.choice(lowercase))
        password.append(random.choice(uppercase))
        password.append(random.choice(digits))
    elif complexity_level == 'complex':
        characters = lowercase + uppercase + digits + special_characters
        password.append(random.choice(lowercase))
        password.append(random.choice(uppercase))
        password.append(random.choice(digits))
        password.append(random.choice(special_characters))
    else:
        raise ValueError("Invalid complexity level. Choose from 'simple', 'medium', or 'complex'.")

    while len(password) < length:
        password.append(random.choice(characters))

    random.shuffle(password)
    return ''.join(password)

In [ ]:
#Password Testing Functionality

In [3]:
# Function to test password strength
def test_password_strength(password):
    feedback = []

    # Length criterion
    if len(password) >= 12:
        feedback.append("Good length.")
    elif 8 <= len(password) < 12:
        feedback.append("Moderate length.")
    else:
        feedback.append("Too short.")

    # Character diversity
    has_lowercase = bool(re.search(r'[a-z]', password))
    has_uppercase = bool(re.search(r'[A-Z]', password))
    has_digit = bool(re.search(r'\d', password))
    has_special = bool(re.search(r'[!@#$%^&*(),.?":{}|<>]', password))

    if has_lowercase and has_uppercase and has_digit and has_special:
        feedback.append("Good character diversity.")
    else:
        feedback.append("Needs more character diversity.")

    # Predictability (Avoiding common patterns)
    common_words = ['password', '1234', 'qwerty', 'admin', 'welcome']
    if any(word in password.lower() for word in common_words):
        feedback.append("Too predictable, avoid common words or patterns.")
    else:
        feedback.append("No common patterns detected.")

    # Determine overall strength based on criteria
    if len(password) >= 12 and has_lowercase and has_uppercase and has_digit and has_special:
        strength = "Strong"
    elif len(password) >= 8 and (has_lowercase or has_uppercase) and (has_digit or has_special):
        strength = "Moderate"
    else:
        strength = "Weak"

    return strength, feedback


In [ ]:
#GUI Application

In [4]:

# Main Window Class
class PasswordApp(QWidget):
    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):
        # Set the window properties
        self.setWindowTitle('Password Generator and Tester')
        self.setGeometry(100, 100, 400, 400)

        # Set the stylesheet for colors
        self.setStyleSheet("""
            QWidget {
                background-color: #2B3A42;  /* Dark background color */
                color: #FFFFFF;  /* White text */
            }
            QLabel {
                font-size: 16px;
                font-weight: bold;
            }
            QComboBox, QLineEdit, QTextEdit {
                background-color: #FFFFFF;
                color: #2B3A42;
                border: 1px solid #FFA500;
                padding: 5px;
                font-size: 14px;
            }
            QPushButton {
                background-color: #FFA500;
                color: #2B3A42;
                border: none;
                padding: 8px;
                font-weight: bold;
                font-size: 14px;
            }
            QPushButton:hover {
                background-color: #FF8C00;
            }
            QTextEdit {
                background-color: #FFFFFF;
                color: #2B3A42;
                border: 1px solid #FFA500;
            }
        """)

        # Create layout
        layout = QVBoxLayout()

        # Password Generation Section
        layout.addWidget(QLabel('Password Generation'))

        # Complexity Level Dropdown
        layout.addWidget(QLabel('Complexity Level:'))
        self.complexity_dropdown = QComboBox()
        self.complexity_dropdown.addItems(['simple', 'medium', 'complex'])
        layout.addWidget(self.complexity_dropdown)

        # Password Length Input
        layout.addWidget(QLabel('Password Length (8-16):'))
        self.length_input = QLineEdit()
        layout.addWidget(self.length_input)

        # Generate Password Button
        self.generate_button = QPushButton('Generate Password', self)
        self.generate_button.clicked.connect(self.generate_password_ui)
        layout.addWidget(self.generate_button)

        # Display Generated Password
        self.password_output = QTextEdit()
        self.password_output.setReadOnly(True)
        layout.addWidget(self.password_output)

        # Password Testing Section
        layout.addWidget(QLabel('Password Testing'))

        # Password Input for Testing
        layout.addWidget(QLabel('Enter Password:'))
        self.password_test_input = QLineEdit()
        self.password_test_input.setEchoMode(QLineEdit.Password)
        layout.addWidget(self.password_test_input)

        # Test Password Button
        self.test_button = QPushButton('Test Password', self)
        self.test_button.clicked.connect(self.test_password_ui)
        layout.addWidget(self.test_button)

        # Display Password Strength Feedback
        self.test_output = QTextEdit()
        self.test_output.setReadOnly(True)
        layout.addWidget(self.test_output)

        # Clear Button
        self.clear_button = QPushButton('Clear', self)
        self.clear_button.clicked.connect(self.clear_output)
        layout.addWidget(self.clear_button)

        # Set layout to the window
        self.setLayout(layout)

    # Clear output fields for a fresh start
    def clear_output(self):
        self.password_output.clear()
        self.password_test_input.clear()
        self.test_output.clear()

    # Generate Password and display in the GUI
    def generate_password_ui(self):
        try:
            complexity = self.complexity_dropdown.currentText()
            length = int(self.length_input.text())
            password = generate_password(complexity, length)
            self.password_output.setText(f"Generated Password: {password}")
        except ValueError as ve:
            QMessageBox.warning(self, 'Error', str(ve))

     # Test password strength and display feedback in the GUI
    def test_password_ui(self):
        password = self.password_test_input.text()
        if not password:
            QMessageBox.warning(self, 'Error', 'Please enter a password to test.')
            return
        
        strength, feedback = test_password_strength(password)
        feedback_text = "\n".join(feedback)
        self.test_output.setText(f"Password Strength: {strength}\nFeedback:\n{feedback_text}")


# Main execution block
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = PasswordApp()
    ex.show()
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\maari\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
